# SciPy Optimization

unconstrained optimization, global optimization, constrained optimization, SVM-style classification, curve fitting, activation fitting, root finding, and systems of nonlinear equations. Each topic includes practical examples with realistic data and clear numerical outputs.

In [1]:
import numpy as np
from scipy import optimize

np.set_printoptions(precision=4, suppress=True)

def r2_score(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((np.asarray(y_true) - np.asarray(y_pred)) ** 2))

## 1. Unconstrained Minimization
Unconstrained minimization finds parameter values that minimize an objective when no explicit bounds or constraints are applied. It is common in model fitting and pricing decisions.

In [2]:
rng = np.random.default_rng(42)
temperature = np.linspace(10, 35, 120)
energy = 48.0 + 2.35 * temperature + rng.normal(0, 9, size=temperature.size)

def mse(params, x, y):
    b0, b1 = params
    pred = b0 + b1 * x
    return np.mean((pred - y) ** 2)

def mse_grad(params, x, y):
    b0, b1 = params
    err = (b0 + b1 * x) - y
    n = y.size
    return np.array([2.0 * err.sum() / n, 2.0 * np.dot(err, x) / n])

bfgs_fit = optimize.minimize(mse, x0=[0.0, 0.0], args=(temperature, energy), jac=mse_grad, method="BFGS")
nelder_fit = optimize.minimize(mse, x0=[0.0, 0.0], args=(temperature, energy), method="Nelder-Mead")

print("Example 1: Energy demand linear model")
print(f"BFGS parameters: intercept={bfgs_fit.x[0]:.3f}, slope={bfgs_fit.x[1]:.3f}, MSE={bfgs_fit.fun:.3f}")
print(f"Nelder-Mead parameters: intercept={nelder_fit.x[0]:.3f}, slope={nelder_fit.x[1]:.3f}, MSE={nelder_fit.fun:.3f}")

def negative_profit(price):
    demand = 1400.0 - 12.0 * price
    return -((price - 40.0) * demand)

price_solution = optimize.minimize_scalar(negative_profit, bounds=(40.0, 100.0), method="bounded")
best_price = price_solution.x
best_demand = 1400.0 - 12.0 * best_price
best_profit = -price_solution.fun

print()
print("Example 2: Retail pricing")
print(f"Optimal price: ${best_price:.2f}")
print(f"Expected units sold: {best_demand:.1f}")
print(f"Expected profit: ${best_profit:.2f}")

Example 1: Energy demand linear model
BFGS parameters: intercept=50.196, slope=2.228, MSE=48.699
Nelder-Mead parameters: intercept=50.196, slope=2.228, MSE=48.699

Example 2: Retail pricing
Optimal price: $78.33
Expected units sold: 460.0
Expected profit: $17633.33


## 2. Global Optimization
Global optimization searches for the best solution across the entire landscape and is useful when many local minima exist.

In [3]:
def rastrigin(x):
    x = np.asarray(x)
    return 10 * x.size + np.sum(x ** 2 - 10 * np.cos(2 * np.pi * x))

bounds_rastrigin = [(-5.12, 5.12), (-5.12, 5.12)]
local_rastrigin = optimize.minimize(rastrigin, x0=[3.5, 3.5], method="L-BFGS-B", bounds=bounds_rastrigin)
global_rastrigin = optimize.differential_evolution(rastrigin, bounds_rastrigin, seed=42, maxiter=250, polish=True)

print("Example 1: Rastrigin function")
print(f"Local search value: {local_rastrigin.fun:.6f} at {local_rastrigin.x}")
print(f"Differential evolution value: {global_rastrigin.fun:.6f} at {global_rastrigin.x}")

def himmelblau(x):
    x1, x2 = np.asarray(x)
    return (x1 ** 2 + x2 - 11) ** 2 + (x1 + x2 ** 2 - 7) ** 2

starts = np.array([[5.5, 5.5], [-5.0, 5.0], [0.5, -4.5]])
local_himmel = [
    optimize.minimize(himmelblau, x0=s, method="L-BFGS-B", bounds=[(-6, 6), (-6, 6)])
    for s in starts
]
annealed_himmel = optimize.dual_annealing(himmelblau, bounds=[(-6, 6), (-6, 6)], seed=42, maxiter=300)

print()
print("Example 2: Himmelblau function")
print("Local search objective values:", [round(r.fun, 6) for r in local_himmel])
print(f"Dual annealing value: {annealed_himmel.fun:.6f} at {annealed_himmel.x}")

Example 1: Rastrigin function
Local search value: 0.000000 at [-0. -0.]
Differential evolution value: 0.000000 at [ 0. -0.]

Example 2: Himmelblau function
Local search objective values: [np.float64(0.0), np.float64(0.0), np.float64(0.0)]
Dual annealing value: 0.000000 at [-3.7793 -3.2832]


## 3. Constrained Optimization
Constrained optimization handles business or engineering limits while still finding the best objective value.

In [4]:
expected_returns = np.array([0.08, 0.12, 0.10, 0.15, 0.06])
cov_matrix = np.array([
    [0.04, 0.01, 0.02, 0.01, 0.005],
    [0.01, 0.09, 0.03, 0.02, 0.01],
    [0.02, 0.03, 0.06, 0.015, 0.008],
    [0.01, 0.02, 0.015, 0.16, 0.02],
    [0.005, 0.01, 0.008, 0.02, 0.02]
])

def portfolio_variance(weights):
    return weights @ cov_matrix @ weights

def portfolio_grad(weights):
    return 2.0 * cov_matrix @ weights

portfolio_constraints = [
    {"type": "eq", "fun": lambda w: np.sum(w) - 1.0},
    {"type": "ineq", "fun": lambda w: w @ expected_returns - 0.10}
]
portfolio_bounds = [(0.0, 0.6) for _ in range(5)]
portfolio_start = np.full(5, 0.2)
portfolio_solution = optimize.minimize(
    portfolio_variance,
    x0=portfolio_start,
    jac=portfolio_grad,
    method="SLSQP",
    bounds=portfolio_bounds,
    constraints=portfolio_constraints
)

assets = np.array(["Stock A", "Stock B", "Stock C", "Stock D", "Bond"])
weights = portfolio_solution.x

print("Example 1: Portfolio optimization")
print(f"Success: {portfolio_solution.success}")
print("Weights:", dict(zip(assets, np.round(weights, 4))))
print(f"Expected return: {weights @ expected_returns:.4f}")
print(f"Portfolio risk (std): {np.sqrt(portfolio_variance(weights)):.4f}")

def negative_monthly_profit(x):
    standard_units, premium_units = x
    return -(40.0 * standard_units + 65.0 * premium_units)

production_constraints = [
    {"type": "ineq", "fun": lambda x: 800.0 - (2.0 * x[0] + 4.0 * x[1])},
    {"type": "ineq", "fun": lambda x: 500.0 - (1.0 * x[0] + 1.5 * x[1])}
]
production_bounds = [(0.0, 260.0), (0.0, 180.0)]
production_solution = optimize.minimize(
    negative_monthly_profit,
    x0=[120.0, 80.0],
    method="SLSQP",
    bounds=production_bounds,
    constraints=production_constraints
)

standard_units, premium_units = production_solution.x

print()
print("Example 2: Production planning")
print(f"Standard units: {standard_units:.1f}")
print(f"Premium units: {premium_units:.1f}")
print(f"Monthly profit: ${-production_solution.fun:.2f}")

Example 1: Portfolio optimization
Success: True
Weights: {np.str_('Stock A'): np.float64(0.2629), np.str_('Stock B'): np.float64(0.2126), np.str_('Stock C'): np.float64(0.1531), np.str_('Stock D'): np.float64(0.1763), np.str_('Bond'): np.float64(0.1952)}
Expected return: 0.1000
Portfolio risk (std): 0.1584

Example 2: Production planning
Standard units: 260.0
Premium units: 70.0
Monthly profit: $14950.00


## 4. SVM-like Classification
This approach minimizes a regularized hinge-loss objective, which gives a maximum-margin style linear classifier.

In [5]:
def hinge_objective(params, X, y, c_pos=1.0, c_neg=1.0):
    w = params[:-1]
    b = params[-1]
    margins = y * (X @ w + b)
    sample_weights = np.where(y > 0, c_pos, c_neg)
    hinge = np.maximum(0.0, 1.0 - margins)
    return 0.5 * np.dot(w, w) + np.sum(sample_weights * hinge)

def linear_predict(params, X):
    scores = X @ params[:-1] + params[-1]
    return np.where(scores >= 0.0, 1.0, -1.0)

rng = np.random.default_rng(7)
X_neg = rng.normal(loc=[-1.5, -1.0], scale=[1.0, 0.9], size=(120, 2))
X_pos = rng.normal(loc=[1.3, 1.5], scale=[0.9, 1.0], size=(120, 2))
X_balanced = np.vstack([X_neg, X_pos])
y_balanced = np.hstack([-np.ones(len(X_neg)), np.ones(len(X_pos))])

svm_balanced = optimize.minimize(
    hinge_objective,
    x0=np.zeros(3),
    args=(X_balanced, y_balanced, 1.0, 1.0),
    method="L-BFGS-B"
)
pred_balanced = linear_predict(svm_balanced.x, X_balanced)
acc_balanced = np.mean(pred_balanced == y_balanced)

print("Example 1: Balanced customer churn classification")
print(f"Weights: {np.round(svm_balanced.x[:-1], 4)}, bias: {svm_balanced.x[-1]:.4f}")
print(f"Training accuracy: {acc_balanced:.4f}")

X_normal = rng.normal(loc=[0.0, 0.0], scale=[1.1, 1.0], size=(450, 2))
X_fraud = rng.normal(loc=[2.6, 2.2], scale=[0.8, 0.7], size=(45, 2))
X_imb = np.vstack([X_normal, X_fraud])
y_imb = np.hstack([-np.ones(len(X_normal)), np.ones(len(X_fraud))])

svm_plain = optimize.minimize(hinge_objective, x0=np.zeros(3), args=(X_imb, y_imb, 1.0, 1.0), method="L-BFGS-B")
svm_weighted = optimize.minimize(hinge_objective, x0=np.zeros(3), args=(X_imb, y_imb, 8.0, 1.0), method="L-BFGS-B")

pred_plain = linear_predict(svm_plain.x, X_imb)
pred_weighted = linear_predict(svm_weighted.x, X_imb)

def recall(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == -1))
    return tp / (tp + fn)

print()
print("Example 2: Imbalanced fraud screening")
print(f"Plain model recall for fraud class: {recall(y_imb, pred_plain):.4f}")
print(f"Class-weighted model recall for fraud class: {recall(y_imb, pred_weighted):.4f}")

Example 1: Balanced customer churn classification
Weights: [1.5644 1.0158], bias: 0.1983
Training accuracy: 0.9833

Example 2: Imbalanced fraud screening
Plain model recall for fraud class: 0.8667
Class-weighted model recall for fraud class: 0.9778


## 5. Curve Fitting
Curve fitting estimates unknown parameters of a model from observed data. It is widely used in growth forecasting and equipment degradation analysis.

In [6]:
rng = np.random.default_rng(21)
months = np.arange(0, 25)

def logistic_growth(t, K, r, P0):
    return K / (1.0 + ((K - P0) / P0) * np.exp(-r * t))

users_true = logistic_growth(months, K=20000.0, r=0.24, P0=1200.0)
users_obs = users_true + rng.normal(0, 180, size=months.size)

fit_logistic, _ = optimize.curve_fit(
    logistic_growth,
    months,
    users_obs,
    p0=[15000.0, 0.2, 900.0],
    bounds=([5000.0, 0.01, 100.0], [50000.0, 1.0, 5000.0])
)
users_pred = logistic_growth(months, *fit_logistic)

print("Example 1: App user growth forecasting")
print(f"Fitted K={fit_logistic[0]:.2f}, r={fit_logistic[1]:.4f}, P0={fit_logistic[2]:.2f}")
print(f"RMSE: {rmse(users_obs, users_pred):.2f}")

def capacity_curve(cycles, a, k, c0):
    return a * np.exp(-k * cycles) + c0

cycle_count = np.arange(0, 801, 20)
capacity_true = capacity_curve(cycle_count, a=24.0, k=0.0028, c0=76.0)
capacity_obs = capacity_true + rng.normal(0, 0.35, size=cycle_count.size)

fit_capacity, _ = optimize.curve_fit(
    capacity_curve,
    cycle_count,
    capacity_obs,
    p0=[20.0, 0.002, 75.0],
    bounds=([5.0, 0.0001, 60.0], [40.0, 0.01, 90.0])
)
capacity_pred = capacity_curve(cycle_count, *fit_capacity)

ratio = (80.0 - fit_capacity[2]) / fit_capacity[0]
if fit_capacity[0] > 0 and fit_capacity[1] > 0 and 0 < ratio < 1:
    cycles_at_80 = -np.log(ratio) / fit_capacity[1]
else:
    cycles_at_80 = np.nan

print()
print("Example 2: Battery capacity degradation")
print(f"Fitted parameters: a={fit_capacity[0]:.4f}, k={fit_capacity[1]:.6f}, c0={fit_capacity[2]:.4f}")
print(f"RMSE: {rmse(capacity_obs, capacity_pred):.4f}")
print(f"Estimated cycles at 80% capacity: {cycles_at_80:.1f}")

Example 1: App user growth forecasting
Fitted K=19945.40, r=0.2442, P0=1158.73
RMSE: 194.18

Example 2: Battery capacity degradation
Fitted parameters: a=24.1415, k=0.002850, c0=76.0548
RMSE: 0.2812
Estimated cycles at 80% capacity: 635.6


## 6. Activation Function Fitting
Activation fitting is useful when you want a smooth function that matches observed response data in neural or probabilistic systems.

In [7]:
rng = np.random.default_rng(99)
x = np.linspace(-4, 4, 160)

def sigmoid(x, L, x0, k):
    z = np.clip(k * (x - x0), -60, 60)
    return L / (1.0 + np.exp(-z))

def tanh_activation(x, a, b, c, d):
    return a * np.tanh(b * (x - c)) + d

def softplus(x, a, b, c, d):
    z = np.clip(b * (x - c), -60, 60)
    return a * np.log1p(np.exp(z)) + d

y_true = sigmoid(x, 1.0, 0.2, 1.6)
y_obs = np.clip(y_true + rng.normal(0, 0.04, size=x.size), 0.0, 1.2)

fit_sigmoid, _ = optimize.curve_fit(sigmoid, x, y_obs, p0=[1.0, 0.0, 1.0], bounds=([0.5, -2.0, 0.2], [1.5, 2.0, 4.0]))
fit_tanh, _ = optimize.curve_fit(tanh_activation, x, y_obs, p0=[0.5, 1.0, 0.0, 0.5], bounds=([0.1, 0.1, -2.0, 0.0], [1.5, 4.0, 2.0, 1.0]))
fit_softplus, _ = optimize.curve_fit(softplus, x, y_obs, p0=[0.3, 1.0, 0.0, 0.0], bounds=([0.05, 0.1, -2.0, -0.2], [2.0, 4.0, 2.0, 0.8]))

pred_sigmoid = sigmoid(x, *fit_sigmoid)
pred_tanh = tanh_activation(x, *fit_tanh)
pred_softplus = softplus(x, *fit_softplus)

print("Example 1: Noisy neuron response fitting")
print(f"Sigmoid R^2: {r2_score(y_obs, pred_sigmoid):.4f}")
print(f"Tanh R^2: {r2_score(y_obs, pred_tanh):.4f}")
print(f"Softplus R^2: {r2_score(y_obs, pred_softplus):.4f}")

scores = np.linspace(-3, 3, 140)
prob_true = sigmoid(scores, 1.0, 0.4, 2.3)
prob_obs = np.clip(prob_true + rng.normal(0, 0.03, size=scores.size), 0.0, 1.0)
fit_calibration, _ = optimize.curve_fit(
    sigmoid,
    scores,
    prob_obs,
    p0=[1.0, 0.0, 1.0],
    bounds=([0.8, -2.0, 0.1], [1.2, 2.0, 5.0])
)

print()
print("Example 2: Probability calibration from model scores")
print(f"Fitted L={fit_calibration[0]:.4f}, threshold={fit_calibration[1]:.4f}, slope={fit_calibration[2]:.4f}")
print(f"Score near 50% probability: {fit_calibration[1]:.4f}")

Example 1: Noisy neuron response fitting
Sigmoid R^2: 0.9921
Tanh R^2: 0.9923
Softplus R^2: 0.9443

Example 2: Probability calibration from model scores
Fitted L=0.9919, threshold=0.3813, slope=2.2798
Score near 50% probability: 0.3813


## 7. Root Finding
Root finding solves equations of the form f(x)=0, which is useful for equilibrium analysis and reverse engineering unknown rates.

In [8]:
def demand(price):
    return 1100.0 - 45.0 * price + 0.4 * price ** 2

def supply(price):
    return 150.0 + 28.0 * price - 0.1 * price ** 2

def excess_demand(price):
    return demand(price) - supply(price)

price_brent = optimize.brentq(excess_demand, 0.0, 40.0)
price_newton = optimize.newton(excess_demand, x0=10.0)
quantity_eq = demand(price_brent)

print("Example 1: Market equilibrium")
print(f"Equilibrium price (brentq): {price_brent:.4f}")
print(f"Equilibrium price (newton): {price_newton:.4f}")
print(f"Equilibrium quantity: {quantity_eq:.4f}")

principal = 300000.0
months = 360
true_monthly_rate = 0.0042
payment = principal * true_monthly_rate * (1 + true_monthly_rate) ** months / ((1 + true_monthly_rate) ** months - 1)

def payment_error(monthly_rate):
    growth = (1 + monthly_rate) ** months
    return principal * monthly_rate * growth / (growth - 1) - payment

rate_brent = optimize.brentq(payment_error, 1e-6, 0.03)
rate_newton = optimize.newton(payment_error, x0=0.005)

print()
print("Example 2: Implied loan interest rate")
print(f"Monthly rate from brentq: {rate_brent:.6f}")
print(f"Monthly rate from newton: {rate_newton:.6f}")
print(f"Annual nominal rate: {12 * rate_brent:.4%}")

Example 1: Market equilibrium
Equilibrium price (brentq): 14.4423
Equilibrium price (newton): 14.4423
Equilibrium quantity: 533.5273

Example 2: Implied loan interest rate
Monthly rate from brentq: 0.004200
Monthly rate from newton: 0.004200
Annual nominal rate: 5.0400%


## 8. System of Nonlinear Equations
Systems of nonlinear equations solve multiple coupled unknowns together. This is common when variables depend on each other through nonlinear relationships.

In [9]:
def traffic_system(vars_):
    toll, flow = vars_
    return [
        flow - 4000.0 * np.exp(-0.015 * toll),
        toll - (2.0 + 0.0008 * flow ** 2)
    ]

traffic_solution = optimize.root(traffic_system, x0=[20.0, 1500.0], method="hybr")

toll_star, flow_star = traffic_solution.x

print("Example 1: Toll and traffic equilibrium")
print(f"Success: {traffic_solution.success}")
print(f"Toll: {toll_star:.4f}")
print(f"Traffic flow: {flow_star:.4f}")
print(f"Residual norm: {np.linalg.norm(traffic_solution.fun):.2e}")

true_location = np.array([2.0, 3.0])
anchor_a = np.array([0.0, 0.0])
anchor_b = np.array([4.0, 1.0])
distance_a = np.linalg.norm(true_location - anchor_a)
distance_b = np.linalg.norm(true_location - anchor_b)

def location_system(vars_):
    x_pos, y_pos = vars_
    return [
        (x_pos - anchor_a[0]) ** 2 + (y_pos - anchor_a[1]) ** 2 - distance_a ** 2,
        (x_pos - anchor_b[0]) ** 2 + (y_pos - anchor_b[1]) ** 2 - distance_b ** 2
    ]

location_solution = optimize.root(location_system, x0=[1.0, 2.0], method="hybr")

print()
print("Example 2: 2D sensor localization")
print(f"Estimated location: {location_solution.x}")
print(f"True location: {true_location}")
print(f"Residual norm: {np.linalg.norm(location_solution.fun):.2e}")

Example 1: Toll and traffic equilibrium
Success: True
Toll: 148.9122
Traffic flow: 428.5326
Residual norm: 8.99e-12

Example 2: 2D sensor localization
Estimated location: [2. 3.]
True location: [2. 3.]
Residual norm: 0.00e+00
